# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [54]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [55]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [56]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 


csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [57]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Completing the Apache Cassandra coding portion of the project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [58]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Creating Keyspace

In [59]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1}""")

#### Set Keyspace

In [60]:
session.set_keyspace('udacity')

## Task 1: Getting artist, song title and song's length.

### In this query sessionId and itemInSession are considered as a primary key. Data needs to be find for a sessionId = 338, and itemInSession = 4.


In [63]:
query1 = "CREATE TABLE IF NOT EXISTS song_for_session "
query1 = query1 + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"

In [64]:
session.execute(query1)

### Writing code to insert values for each column.

In [65]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query1 = "INSERT INTO song_for_session (sessionId, itemInSession, artist, song, length)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Using SELECT to verify that the data have been inserted into each table

In [66]:
query1 = "SELECT artist, song, length from song_for_session WHERE sessionId=338 AND itemInSession=4"
rows = session.execute(query1)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Task 2: Getting name of artist, song (sorted by intemInSession) and user (first and last name).


### Data needs to be find for userid = 10, sessionid = 182.

In [68]:
query2 = "CREATE TABLE IF NOT EXISTS song_session_user "
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query2)


### Writing code to insert values for each column.

In [69]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query2 = "INSERT INTO song_session_user (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))
                    

### Using SELECT to verify that the data have been inserted into each table

In [43]:
query = "SELECT artist, song, firstName, lastName FROM song_session_user WHERE userId=10 AND sessionId=182"
rows = session.execute(query)
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Task 3: Getting every user name (first and last).


### Data needs to be find for who listened to the song 'All Hands Against His Own'. In this query userID and song are considered as a Primary key.

In [71]:
query3 = "CREATE TABLE IF NOT EXISTS user_song "
query3 = query3 + "(song text, userId int,firstName text, lastName text, PRIMARY KEY (song, userId))"
session.execute(query3)


                    

### Writing code to insert values for each column.

In [72]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query3 = "INSERT INTO user_song (song, userId, firstName, lastName)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

### Using SELECT to verify that the data have been inserted into each table

In [47]:
query3 = "SELECT firstName, lastName FROM user_song WHERE song='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [49]:
session.execute("drop table song_for_session")
session.execute("drop table song_session_user")
session.execute("drop table user_song")

### Closing the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()